##  Data preparation 

### Necessary imports

In [1]:
import pandas as pd
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, StandardScaler, FunctionTransformer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import KNeighborsRegressor

import warnings
warnings.simplefilter('ignore')

### Loading dataset

In [2]:
pd.set_option("display.max_columns", 25)
df = pd.read_csv('IT Salary Survey EU  2020.csv')

There are too many features in the dataset, let's choose some of them.

In [3]:
features = ['Age', 'Gender', 'City', 'Position', 'Exp', 'Level',
            'Salary', 'Empl_status', 'Contract_duration', 'Company_size']
df = df[features]
df.head()

,Age,Gender,City,Position,Exp,Level,Salary,Empl_status,Contract_duration,Company_size
0,26.0,Male,Munich,Software Engineer,5,Senior,80000.0,Full-time employee,Unlimited contract,51-100
1,26.0,Male,Berlin,Backend Developer,7,Senior,80000.0,Full-time employee,Unlimited contract,101-1000
2,29.0,Male,Berlin,Software Engineer,12,Lead,120000.0,Self-employed (freelancer),Temporary contract,101-1000
3,28.0,Male,Berlin,Frontend Developer,4,Junior,54000.0,Full-time employee,Unlimited contract,51-100
4,37.0,Male,Berlin,Backend Developer,17,Senior,62000.0,Full-time employee,Unlimited contract,101-1000


We'll predict salary of IT specialists:

In [4]:
target = df.Salary
df.drop(columns=['Salary'])
df, df_test, target, target_test = train_test_split(df, target, test_size=0.2, random_state=1)

### Defining custom classes for pipelines

In [5]:
class FeatureSelector(BaseEstimator, TransformerMixin):
    """
    The class provides basic functionality for retrieving
    a subset of columns from the dataset.
    """
    
    def __init__(self, feature_names):
        """
        Initialize class instance by setting
        a list of columns to retrieve from the dataset.
        """
        BaseEstimator.__init__(self)
        TransformerMixin.__init__(self)
        self.feature_names = feature_names
        
    def fit(self, X, y=None):
        """
        Fit FeatureSelector to X, but really do nothing.
        Return self.
        """
        return self
    
    def transform(self, X, y=None):
        """
        Transform X using feature selection. 
        Return column-subset of X.
        """
        return X[self.feature_names]

In [6]:

class ColumnTranslation:
    """
    The class is used to store information about the conversion of a single column.
    """
    
    def __init__(self, column_name, to_save, default='Other'):
        """
        Initialize ColumnTranslation instance.
        
        Parameters
        ----------
        column_name: str, 
            name of column to translate.
        to_save: list, 
            list of values whose translation is not required.
        default: str, 
            value to be written to the column if it is not in the to_save list.
        """
        self.column_name = column_name
        self.to_save = to_save
        self.default = default

In [7]:
class Translator(BaseEstimator, TransformerMixin):
    """
    The class provides functionality for translating column values 
    to a defined range of values.
    """
    
    def __init__(self, translations):
        """
        Initialize class instance.
        
        Parameters
        ----------
        translations: list of ColumnTranslation's, 
            object that provides column_name, to_save and default properties.
        """
        BaseEstimator.__init__(self)
        TransformerMixin.__init__(self)
        self.translations = translations
        
    def fit(self, X, y=None):
        """
        Fit Translator to X, but really does nothing.
        Return self.
        """
        return self
    
    def transform(self, X, y=None):
        """
        Transform X.
        For each column from the list `translations`: 
        the values from the `to_save` list are kept intact;
        default values are written in the remaining rows.
        Return X.
        """
        for tr in self.translations:
            X[tr.column_name] = X[tr.column_name].fillna(tr.default)
            X[tr.column_name] = X[tr.column_name].astype(str).map(lambda x: x if (x in tr.to_save) else tr.default)
        return X

In [8]:
class ColumnOrder:
    """
    The class is used to store information about the conversion of a single column
    to ordinal column.
    """
    
    def __init__(self, col_name, order):
        """
        Initialize ColumnTranslation instance.
        
        Parameters
        ----------
        col_name: str, 
            name of column to converte to ordinal.
        order: list, 
            list of unique values of the column in custom order. 
        """
        self.col_name = col_name
        self.order = order

In [9]:
# Since I haven't found the way to pass to OrdinalEncoder custom order for 
# string values (not alphabet order) I've defined my own encoder.

class OrdinalTransformer(BaseEstimator, TransformerMixin):
    """
    The class provides functionality for conversion columns into ordinal columns.
    """
    def __init__(self, col_orders):
        """
        Initialize class instance.
        
        Parameters
        ----------
        col_orders: list of ColumnOrder's, 
            object that provides column_name and column values' order properties.
        """
        BaseEstimator.__init__(self)
        TransformerMixin.__init__(self)
        self.col_orders = col_orders
        
    def fit(self, X, y=None):
        """
        Fit Translator to X, but really does nothing.
        Return self.
        """
        return self
    
    def transform(self, X, y=None):
        """
        Transform X.
        For each column from the list 'col_orders': 
        converts each value of the column to the value of 
        its index in list 'order'.
        Return X.
        """
        for col_order in self.col_orders:
            X[col_order.col_name] = X[col_order.col_name].map(lambda s: col_order.order.index(s))
        return X

In [10]:
class ImputerByCategorialColumn(BaseEstimator, TransformerMixin):
    """
    The class provides functionality for filling missing values of target column 
    using another categorial column without missings. Each missing of
    target column fills with mean of values of the same category.
    """
    def __init__(self, col, cat_col):
        """
        Initialize class instance.
        
        Parameters
        ----------
        col: name of column with missing values.
        cat_col: name of categorial column without missings.
        """
        BaseEstimator.__init__(self)
        TransformerMixin.__init__(self)
        self.col = col
        self.cat_col = cat_col
        
    def fit(self, X, y=None):
        """
        Save mean value of target column for each category of categorial column
        in dictionary 'map_'.
        Return self.
        """
        X[self.col] = X[self.col].map(lambda s: str(s).replace(',', '.')).astype(float)
        self.map_ = {category: X[X[self.cat_col] == category][self.col].mean() \
                     for category in X[self.cat_col].unique()}
        return self
    
    def transform(self, X, y=None):
        """
        Transform X.
        Each missing of column 'col' fills with mean of values of the 
        same category.
        Return X.
        """
        X[self.col] = X[self.col].map(lambda s: str(s).replace(',', '.')).astype(float)
        for ind in X.index:
            entry = X.loc[ind, self.col]
            is_na = np.isnan(entry)
            X.loc[ind, self.col] = entry if (not is_na) else self.map_[X.loc[ind, self.cat_col]]
        return X

In [11]:
class RegressionImputer(BaseEstimator, TransformerMixin):
    """
    The class provides functionality for filling missing values of target column 
    predicting them using regression model and columns without missings.
    """
    def __init__(self, target_col, cols):
        """
        Initialize class instance.
        
        Parameters
        ----------
        target_col: name of column with missing values.
        cols: list of names of columns without missings.
        """
        BaseEstimator.__init__(self)
        TransformerMixin.__init__(self)
        self.target_col = target_col
        self.cols = cols
        
    def fit(self, X, y=None):
        """
        Fit regression model with features 'cols' and target 'target_col'.
        Return self.
        """
        self.model = LinearRegression()
        X = X[X[self.target_col].notna()]
        self.model.fit(X[self.cols], X[self.target_col])
        return self
    
    def transform(self, X, y=None):
        """
        Transform X.
        Each missing in target column fills with predicted value.
        Return X.
        """
        for ind in X.index:
            X.loc[ind, self.target_col] = X.loc[ind, self.target_col] \
            if (not np.isnan(X.loc[ind, self.target_col])) \
            else self.model.predict([X.loc[ind, self.cols]])
        return X

### Constructing pipelines

As we use KNN as a model, all pipelines will scale values.

First pipeline imputes columns with most frequent value, as these columns are more than 90% filled with one value and there are very few of missings. As number of categories is small (less than 3) it's reasonable to use OneHotEncoder.

In [12]:
translate_impute_ohe_pipeline = Pipeline(
    steps=[
        ('feature_selector', FeatureSelector(['Gender', 'Empl_status', 'Contract_duration'])),
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('pandarizer', FunctionTransformer(
            lambda x: pd.DataFrame(x, columns = ['Gender', 'Empl_status', 'Contract_duration']))),
        ('translator', Translator([
                          ColumnTranslation(column_name='Gender', 
                                            to_save=['Male', 'Female'], 
                                            default='Male'),
                          ColumnTranslation(column_name='Empl_status', 
                                            to_save=['Full-time employee']),
                          ColumnTranslation(column_name='Contract_duration', 
                                            to_save=['Unlimited contract'])
                        ])),
         ('encoder', OneHotEncoder(sparse=False)),
         ('scaler', StandardScaler())
    ]
)

Second pipeline encodes column with companies' sizes categories to ordinal values.

In [13]:
translate_ord_pipeline = Pipeline(
    steps=[
           ('feature_selector', FeatureSelector(['Company_size'])),
           ('translator', Translator([ColumnTranslation(column_name='Company_size', 
                                               to_save=['up to 10', '11-50', 
                                                        '51-100', '101-1000', 
                                                        '1000+'], 
                                               default='101-1000')])),
           ('to_ordinal_transformer', OrdinalTransformer(
               col_orders=[
                           ColumnOrder(col_name='Company_size',
                                       order=['up to 10', '11-50', '51-100',
                                              '101-1000', '1000+'])
                          ])),
           ('scaler', StandardScaler())
           ]
)

Third pipeline is similar to first, but it doesn't impute values (number of categories is also small).

In [14]:
translate_ohe_pipeline = Pipeline(
    steps=[
           ('feature_selector', FeatureSelector(['City', 'Position'])),
           ('translator', Translator([ColumnTranslation(column_name='City', 
                                               to_save=['Berlin', 'Munich']),
                             ColumnTranslation(column_name='Position', 
                                               to_save=['Software Engineer', 
                                                        'Backend Developer', 
                                                        'Data Scientist', 
                                                        'Frontend Developer', 
                                                        'QA Engineer'])])),
           ('encoder', OneHotEncoder(sparse=False)),
           ('scaler', StandardScaler())  
    ]  
)

Fourth pipeline is the most interesting one. 
It imputes missings in 'experience' by speialist's level ('junior', 'middle', ...) as 'level' column has almost no missings. 
As soon as 'experience' column has no missings, we predict missing 'ages' by years of experience. 
Finally, it encodes level to ordinal, since the level values are comparable.

In [15]:
translate_regr_pipeline = Pipeline(
    steps=[
        ('feature_selector', FeatureSelector(['Level', 'Exp', 'Age'])),
        ('translator', Translator([ColumnTranslation(column_name='Level', 
                                               to_save=['Junior', 'Middle', 
                                                        'Senior', 'Lead', 
                                                        'Head'], 
                                               default='Junior')])),
        ('imputer_cat_col', ImputerByCategorialColumn(col='Exp', cat_col='Level')),
        ('imputer_regr', RegressionImputer(target_col='Age', cols=['Exp'])),
        ('transformer_to_ord', OrdinalTransformer(
            col_orders=[
                        ColumnOrder(col_name='Level',
                                    order=['Junior', 'Middle', 'Senior',
                                           'Lead', 'Head'])
                       ])),
        ('scaler', StandardScaler())
           
    ]
)


Just compose all features.

In [16]:
full_pipeline = FeatureUnion(transformer_list=[
    ('translate_impute_ohe', translate_impute_ohe_pipeline),
    ('translate_ord', translate_ord_pipeline),
    ('translate_ohe', translate_ohe_pipeline),
    ('translate_regr', translate_regr_pipeline),
])

Make final pipeline, fit on training data, predict target column ('Salary') for test data.

In [17]:
final_pipeline = Pipeline(steps=[
                                 ('data preparation', full_pipeline), 
                                 ('model', KNeighborsRegressor()) 
                                 ]
                          )
final_pipeline.fit(df, target)
print('Real values of salary:')
print(target_test.values[:9])
print('Predicted values of salary:')
print(final_pipeline.predict(df_test)[:9])

Real values of salary:
[ 75000.  54000.  65000. 150000.  67200.  80000.  72000. 120000.  64000.]
Predicted values of salary:
[71480. 61000. 50900. 75600. 68080. 73800. 87000. 69800. 66720.]


The prediction isn't very accurate, but since the task was to prepare data for some model, I think it's ok ^^